In [9]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
FLAGS = None

In [10]:
mnist = input_data.read_data_sets("./MNIST_Data", one_hot=False)

Extracting ./MNIST_Data\train-images-idx3-ubyte.gz
Extracting ./MNIST_Data\train-labels-idx1-ubyte.gz
Extracting ./MNIST_Data\t10k-images-idx3-ubyte.gz
Extracting ./MNIST_Data\t10k-labels-idx1-ubyte.gz


In [11]:
tf.reset_default_graph()
sess = tf.InteractiveSession()

x = tf.placeholder(tf.float32, [None, 784])

x_img = tf.reshape(x, [-1, 28, 28, 1])

with tf.variable_scope('conv1'):
    W = tf.get_variable("W", [10, 10, 1, 32],
        initializer=tf.truncated_normal_initializer(stddev=np.sqrt(2/784)))
    b = tf.get_variable("b", initializer=tf.constant(0.1, shape=[32]))
    
    z = tf.nn.conv2d(x_img, W, strides=[1,1,1,1], padding="SAME") + b
    h1 = tf.nn.relu(z)
    
with tf.variable_scope('pool1'):
    h2 = tf.nn.max_pool(h1, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
    
    
    
with tf.variable_scope('conv2'):
    W = tf.get_variable("W", [5, 5, 32, 64],
        initializer=tf.truncated_normal_initializer(stddev=np.sqrt(2.0/64)))
    b = tf.get_variable("b", initializer=tf.constant(0.1, shape=[64]))
    
    z = tf.nn.conv2d(h2, W, strides=[1,1,1,1], padding="SAME") + b
    h3 = tf.nn.relu(z)
    
with tf.variable_scope('pool2'):
    h4 = tf.nn.max_pool(h3, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")


with tf.variable_scope('fc'):
    flat_size = int(np.prod(h4.shape[1:]))
    h4_flat = tf.reshape(h4, [-1, flat_size])
    
    W = tf.get_variable("W", [flat_size, 1024],
        initializer=tf.truncated_normal_initializer(stddev=np.sqrt(2/flat_size)))
    
    b = tf.get_variable("b", initializer=tf.constant(0.1, shape=[1024]))
    
    h5 = tf.nn.relu(tf.matmul(h4_flat, W) + b)
    
with tf.variable_scope('fc2'):    
    W = tf.get_variable("W", [1024, 10],
        initializer=tf.truncated_normal_initializer(stddev=np.sqrt(2/1024)))
    
    b = tf.get_variable("b", initializer=tf.constant(0.1, shape=[10]))
    
    h6 = tf.matmul(h5, W) + b 
    
y = h6#tf.nn.softmax(logits)

labels = tf.placeholder(tf.int64, [None])

loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=y)
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(loss)
tf.global_variables_initializer().run()


C:\Users\Jacob\Anaconda3\lib\site-packages\tensorflow\python\client\session.py:1711: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [12]:
for i in range(1000):
    batch_x, batch_y = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_x, labels: batch_y})
    
    if i%100 == 0:
        correct_prediction = tf.equal(tf.argmax(y, 1), labels)
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        print(sess.run(
          accuracy, feed_dict={
              x: mnist.test.images,
              labels: mnist.test.labels
          }))

ResourceExhaustedError: OOM when allocating tensor with shape[10000,32,28,28] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: conv1/Conv2D = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](Reshape, conv1/W/read)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: Mean/_9 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_58_Mean", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocati

In [8]:
correct_prediction = tf.equal(tf.argmax(y, 1), labels)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(
  accuracy, feed_dict={
      x: mnist.test.images,
      labels: mnist.test.labels
  }))

ResourceExhaustedError: OOM when allocating tensor with shape[10000,32,28,28] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: conv1/Conv2D = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](Reshape, conv1/W/read)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: Mean_1/_15 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_58_Mean_1", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current all